In [ ]:
# use this notebook to test the data augmentation functions, and to generate ECG photograph datasets (see bottommost cell)

In [ ]:
# imports

from data_augmentation import *
from IPython.core.display_functions import display
import json
import math
import numpy as np
from PIL import Image
import os

In [ ]:
# tips for finding sensible default parameters for the parameter distributions of get_random_ecg_photo:
#
# fix all sigmas to 0 except the parameter(s) to vary; downscale image for faster generation time
# go to desmos.com, input Gaussian/Beta PDF formula with parameters to see how likely a given parameter value will be
# for shadows, we can have either a bright shadow (camera flash) or a dark shadow (normal shadow); we use a U-shaped beta distribution to simulate this

In [ ]:
bg_img = get_background_image(3)  # useful for fixing a specific background image

In [ ]:
# experiment with different parameters here

img, label, lead_pos, angle = get_random_ecg_photo(original_image_scaling_factor_mu=1.0,
                                                   original_image_scaling_factor_sd=0.0,
                                                   blur_factor_mu=0.1,
                                                   blur_factor_sd=0.05,
                                                   ecg_paper_scale_mu=1.02,
                                                   ecg_paper_scale_sd=0.02,
                                                   ecg_paper_y_skew_mu=1.0,
                                                   ecg_paper_y_skew_sd=0.08,
                                                   rotation_angle_mu=0.0,
                                                   rotation_angle_sd=5.0,
                                                   ecg_paper_relative_translation_x_mu=0.0,
                                                   ecg_paper_relative_translation_x_sd=0.05,
                                                   ecg_paper_relative_translation_y_mu=0.0,
                                                   ecg_paper_relative_translation_y_sd=0.05,
                                                   shadow_color_beta_1=0.3,
                                                   shadow_color_beta_2=0.3,
                                                   shadow_alpha_beta_1=8.0,
                                                   shadow_alpha_beta_2=25.0,
                                                   shadow_relative_start_point_mu=0.0,
                                                   shadow_relative_start_point_sd=0.1,
                                                   shadow_relative_end_point_mu=1.0,
                                                   shadow_relative_end_point_sd=0.1,
                                                   shadow_blur_factor_mu=50.0,
                                                   shadow_blur_factor_sigma=3.0,
                                                   white_noise_p_beta_1=5.0,
                                                   white_noise_p_beta_2=5.0,
                                                   white_noise_sigma_mu=20.0,
                                                   white_noise_sigma_sigma=5.0)
                                                   #background_image=bg_img)
display(img)

In [ ]:
save_dir = os.path.join('datasets', 'ptb_v')
os.makedirs(save_dir, exist_ok=True)

In [ ]:
# dataset generation

import datetime

num_samples_to_generate = 100

for sample_idx in range(num_samples_to_generate):
    photo, data, lead_pos, angle = get_random_ecg_photo()
    scp_codes = data['scp_codes']
    scp_code_str = '_'.join([f'{k}={v}' for k, v in scp_codes.items()])
    rec_date = datetime.datetime.strptime(data['recording_date'], '%Y-%m-%d %H:%M:%S')
    rec_timestamp = (rec_date - datetime.datetime(1970, 1, 1)).total_seconds()
    id_str = str(int(data['patient_id'])) + '_' + str(int(rec_timestamp))
    if len(scp_code_str) > 0:
        scp_code_str = '_' + scp_code_str
    path_no_ext = (('%0' + str(math.ceil(math.log10(num_samples_to_generate))) + 'd_' + id_str + scp_code_str)
                   % sample_idx)
    path_photo = os.path.join(save_dir, path_no_ext + '.jpg')
    photo.convert('RGB').save(path_photo)

    img_width, img_height = photo.size
    with open(os.path.join(save_dir, path_no_ext + '_lead_pos.json'), 'w') as lead_pos_file:
        lead_pos['_file_path'] = path_photo
        lead_pos['_angle'] = angle
        lead_pos['_img_width'] = img_width
        lead_pos['_img_height'] = img_height
        lead_pos_file.write(json.dumps(lead_pos))

In [ ]:
# generation of individual lead pictures

# lead bounding box distortion:

lead_bounding_box_distortion_sigma = 15.0


def get_new_point_pos(pt_x, pt_y, angle_deg, img_width, img_height):
    # credit to https://stackoverflow.com/a/51964802
    center_x = int(img_width / 2)
    center_y = int(img_height / 2)

    angle_rad = (angle_deg / 180.0) * math.pi
    new_px = center_x + int(float(pt_x - center_x) * math.cos(angle_rad)
                            + float(pt_y - center_y) * math.sin(angle_rad))
    new_py = center_y + int(-(float(pt_x - center_x) * math.sin(angle_rad))
                            + float(pt_y - center_y) * math.cos(angle_rad))
    return new_px, new_py

individual_lead_dir = os.path.join(save_dir, 'individual_leads')
os.makedirs(individual_lead_dir, exist_ok=True)

for root, dirs, files in os.walk(save_dir):
    for filename in files:
        if not filename.lower().endswith('.json'):
            continue
        with open(os.path.join(save_dir, filename)) as f:
            content = f.read()
            try:
                lead_pos = json.loads(content)
                _file_path = lead_pos['_file_path']
                angle = lead_pos['_angle']
                img_width = lead_pos['_img_width']
                img_height = lead_pos['_img_height']
            except Exception as e:
                continue

            photo_fn = os.path.basename(_file_path)
            dot_idx = photo_fn.rfind('.')
            photo_ext = photo_fn[dot_idx+1:]
            individual_lead_fn_prefix = os.path.join(individual_lead_dir, photo_fn[:dot_idx])

            img = Image.open(_file_path)
            rot_img = img.rotate(-angle)

            rot_kwargs = {'angle_deg': -angle, 'img_width': img_width, 'img_height': img_height}

            for lead_name, pos in lead_pos.items():
                if lead_name.startswith('_'):
                    continue
                
                tl_x, tl_y, tr_x, tr_y, bl_x, bl_y, br_x, br_y =\
                    [round(p + np.random.normal(0, lead_bounding_box_distortion_sigma)) for p in pos]
                tl_x_rot, tl_y_rot = get_new_point_pos(tl_x, tl_y, **rot_kwargs)
                tr_x_rot, tr_y_rot = get_new_point_pos(tr_x, tr_y, **rot_kwargs)
                bl_x_rot, bl_y_rot = get_new_point_pos(bl_x, bl_y, **rot_kwargs)
                br_x_rot, br_y_rot = get_new_point_pos(br_x, br_y, **rot_kwargs)
                
                leftmost_x   = max(0, min(img_width - 1, min([tl_x_rot, tr_x_rot, bl_x_rot, br_x_rot])))
                topmost_y    = max(0, min(img_height - 1, min([tl_y_rot, tr_y_rot, bl_y_rot, br_y_rot])))
                
                rightmost_x  = max(0, min(img_width - 1, max([tl_x_rot, tr_x_rot, bl_x_rot, br_x_rot])))
                bottommost_y = max(0, min(img_height - 1, max([tl_y_rot, tr_y_rot, bl_y_rot, br_y_rot])))

                lead_img = rot_img.crop((leftmost_x, topmost_y, rightmost_x, bottommost_y))
                
                lead_img_path = f'{individual_lead_fn_prefix}_LEAD={lead_name}.{photo_ext}'
                lead_img.save(lead_img_path)
